# Test Phase Transition at Finite Temperature for Different Benchmark Points

In [1]:
import numpy as np
from cosmoTransitions import generic_potential as gp
from cosmoTransitions import pathDeformation as pd

## Constants
- $m_H$ = 125. GeV
- $v_{EW}$ = 174. GeV

In [2]:
GF = 1.16637e-05
v = 1/(np.sqrt(2*np.sqrt(2)*GF))
mHSM = 125.13

## Transfer physical parameters to bare parameters
$$
\begin{align*}
\mu_H^2 &= \frac{1}{2}(m_h^2 \cos^2 \theta + m_S^2 \sin^2 \theta)\\
\mu_S^2 &= m_h^2 \sin^2 \theta + m_S^2 \cos^2 \theta\\
A &= \frac{(m_h^2 - m_S^2)\sin \theta \cos \theta}{\sqrt{2}v}\\
\lambda &= \frac{m_h^2 \cos^2 \theta + m_S^2 \sin^2 \theta}{4 v^2}
\end{align*}
$$


In [3]:
def muH2(mS, sintheta):
    """\mu_H square"""
    return 0.5*(mHSM**2 * (1-sintheta**2) + mS**2 * sintheta**2)

def muS2(mS, sintheta):
    """\mu_S square"""
    return sintheta**2 * mHSM**2 + (1 - sintheta**2) * mS**2

def A(mS, sintheta):
    """A parameter"""
    nominator = (mHSM**2 - mS**2) * sintheta * np.sqrt(1-sintheta**2)
    denominator = np.sqrt(2) * v
    return nominator/denominator

def lm(mS, sintheta):
    """\lambda parameter"""
    nominator = (1 - sintheta**2)*mHSM**2 + sintheta**2 * mS**2
    return nominator/(4*v**2)

## Define effective potential

In [4]:
class model(gp.generic_potential):
    def init(self, mS, sintheta):
        self.Ndim = 2
        self.Tmax = 100
        self.mS = mS
        self.sintheta = sintheta
        self.lm = lm(self.mS, self.sintheta)
        self.A = A(self.mS,self.sintheta)
        self.muH2 = muH2(self.mS,self.sintheta)
        self.muS2 = muS2(self.mS,self.sintheta)
        self.g = 0.65
        self.gY = 0.36
        self.yt = 0.9945
        self.D = (3*self.g**2 + self.gY**2 + 4*self.yt**2)/16.
        self.E = (2*self.g**3+(self.g**2 + self.gY**2)**(3/2))/(48*np.pi)
        self.cs = 1./3
        self.Deff = self.D - self.cs * self.A**2/(4.*self.muS2)
        self.lmeff = self.lm - self.A**2/(2*self.muS2)
        self.T0 = np.sqrt(0.5*self.muH2 - v**2 * self.A**2 /(2*self.muS2))/np.sqrt(self.D - self.cs*self.A**2/(4*self.muS2))
        self.Tc = self.T0*np.sqrt((self.Deff * self.lmeff)/(-self.E**2 + self.Deff*self.lmeff))
        self.strength = 2*self.E/self.lmeff

    def Vtot1d(self, X, T, include_radiation = True):
        T = np.asanyarray(T, dtype=float)
        X = np.asanyarray(X, dtype=float)
        T2 = (T*T) + 1e-100
        phi1 = X[...,0]
        y = self.Deff * T2 * phi1**2 - (0.5*self.muH2 - 0.5 * v**2 * self.A**2 / (self.muS2))*phi1**2
        y += - self.E * T * phi1 **3
        y += 0.25 * self.lmeff * phi1**4
        return y



    def Vtot(self, X, T, include_radiation=True):
        T = np.asanyarray(T, dtype=float)
        X = np.asanyarray(X, dtype=float)
        T2 = (T*T)+1e-100
        phi1 = X[...,0]
        phi2 = X[...,1]
        y = self.D * T2 * phi1**2 - 0.5 * self.muH2 * phi1**2
        y += - self.E * T * phi1**3
        y += 0.25 * self.lm * phi1**4
        y += 0.5*self.muS2*phi2**2 - 0.5 * self.A * (phi1**2 + self.cs * T2 - 2 * v**2)*phi2
        return y

    def Spath(self, X, T):
        X = np.asanyarray(X)
        T = np.asanyarray(T)
        phi1 = X[...,0]
        T2 = (T*T) + 1e-100
        return 0.5*self.A*(phi1**2 + self.cs * T2 - 2 * v**2)/self.muS2



## Test Phase Transition Computation

In [17]:
m=model(5, 0.245)

In [18]:
m.findAllTransitions()

Tracing phase starting at x = [2.46220571e+02 1.11443201e-05] ; t = 0.0
Tracing minimum up
traceMinimum t0 = 0
........................................................
Tracing phase starting at x = [-9.09316425e-06 -4.73245510e+02] ; t = 22.219845082611194
Tracing minimum down
traceMinimum t0 = 22.2198
.............................................................
Tracing minimum up
traceMinimum t0 = 22.2198
........................................................................................................................................................................................................................................................
Tunneling from phase 1 to phase 0 at T=18.00555
high_vev = [-1.44325853e-05 -4.73687816e+02]
low_vev = [  99.51175867 -396.17613174]
Path deformation converged. 28 steps. fRatio = 1.78249e-02
Path deformation converged. 18 steps. fRatio = 1.56223e-02
Path deformation converged. 7 steps. fRatio = 4.91794e-03
Path deformation converged. 1 s

[{'low_vev': array([  95.67536992, -402.0320353 ]),
  'high_vev': array([-7.50892750e-06, -4.73682457e+02]),
  'Tnuc': 18.06255999899942,
  'low_phase': 0,
  'high_phase': 1,
  'instanton': fullTunneling_rval(profile1D=Profile1D(R=array([0.00000000e+00, 6.72006611e-05, 1.13787125e-02, 2.26902243e-02,
         3.40017361e-02, 4.53132479e-02, 5.66247598e-02, 6.79362716e-02,
         7.92477834e-02, 9.05592952e-02, 1.01870807e-01, 1.13182319e-01,
         1.24493831e-01, 1.35805342e-01, 1.47116854e-01, 1.58428366e-01,
         1.69739878e-01, 1.81051390e-01, 1.92362902e-01, 2.03674413e-01,
         2.14985925e-01, 2.26297437e-01, 2.37608949e-01, 2.48920461e-01,
         2.60231972e-01, 2.71543484e-01, 2.82854996e-01, 2.94166508e-01,
         3.05478020e-01, 3.16789532e-01, 3.28101043e-01, 3.39412555e-01,
         3.50724067e-01, 3.62035579e-01, 3.73347091e-01, 3.84658602e-01,
         3.95970114e-01, 4.07281626e-01, 4.18593138e-01, 4.29904650e-01,
         4.41216162e-01, 4.52527673e-01, 

In [19]:
print(m.Tc,m.TnTrans[0]['Tnuc'])

18.31545781304192 18.06255999899942


In [8]:
def SoverT(T):
    def V_(x, T=T, V=m.Vtot):
        return V(x,T)
    def dV_(x, T=T, dV=m.gradV):
        return dV(x,T)
    tobj = pd.fullTunneling([m.findMinimum(T=T),[0,m.Spath([0],T)]],V_,dV_)
    return tobj.action/T

In [15]:
def beta_over_H(Tc,Tn):
    "Ridders algorithm"
    if Tc-Tn >=0.002: eps = 0.001
    else: eps=0.99*(Tc-Tn)/2
    dev = (SoverT(Tn-2.*eps) - 8.*SoverT(Tn-eps) + 8.*SoverT(Tn+eps)-SoverT(Tn+2.*eps))/(12.*eps)
    return dev*Tn


In [26]:
def alpha(Tc, Tn):
    if Tc-Tn >=0.002: eps = 0.001
    else: eps=0.99*(Tc-Tn)/2
    def deltaV(T):
        falsev=[0,m.Spath([0],T)]
        truev=m.findMinimum(T=T)
        return m.Vtot(falsev,T)-m.Vtot(truev,T)
    dev = (deltaV(Tn-2*eps) - 8.*deltaV(Tn-eps) + 8.*deltaV(Tn+eps) - deltaV(Tn+2.*eps))/(12.*eps) # derivative of deltaV w.r.t T at Tn
    latent=deltaV(Tn) - 0.25*Tn*dev
    rho_crit = np.pi**2*106.75*Tn**4/30.
    return latent/rho_crit

In [27]:
alpha(m.Tc,m.TnTrans[0]['Tnuc'])

0.12437381924183659

In [20]:
beta_over_H(m.Tc, m.TnTrans[0]['Tnuc'])

Path deformation converged. 26 steps. fRatio = 6.75607e-03
Path deformation converged. 22 steps. fRatio = 1.22278e-02
Path deformation converged. 6 steps. fRatio = 1.28375e-02
Path deformation converged. 1 steps. fRatio = 5.95248e-03
Path deformation converged. 26 steps. fRatio = 6.69512e-03
Path deformation converged. 22 steps. fRatio = 1.15135e-02
Path deformation converged. 5 steps. fRatio = 1.99787e-02
Path deformation converged. 1 steps. fRatio = 1.28016e-02
Path deformation converged. 26 steps. fRatio = 6.57730e-03
Path deformation converged. 22 steps. fRatio = 1.02046e-02
Path deformation converged. 5 steps. fRatio = 1.96952e-02
Path deformation converged. 1 steps. fRatio = 1.26446e-02
Path deformation converged. 26 steps. fRatio = 6.52037e-03
Path deformation converged. 22 steps. fRatio = 9.61288e-03
Path deformation converged. 5 steps. fRatio = 1.95345e-02
Path deformation converged. 1 steps. fRatio = 1.25210e-02


35423.23996669028